Modelo de Recomendación

Se creó un sistema de recomendación basado en el método item-item, aplicando técnicas de aprendizaje automático. Para ello, se cargaron y combinaron los datos de juegos y sus géneros a partir de archivos en formato parquet, normalizando las fechas de lanzamiento de los juegos. Posteriormente, se realizó una selección de los datos, limitando la matriz a un 20% para su uso en la API. Se generó una matriz de similitud basada en el coseno para evaluar la relación entre los juegos, considerando características como la fecha de lanzamiento y los géneros. Se desarrolló una función llamada 'recomendacion_juego_muestreado' que acepta el ID de un juego y devuelve una lista de cinco títulos recomendados que son similares al juego proporcionado, utilizando la matriz de similitud previamente elaborada. Esta función se incorpora en una API a través de un método GET, permitiendo a los usuarios ingresar el ID de un juego y recibir recomendaciones de juegos semejantes. De este modo, los usuarios pueden obtener sugerencias personalizadas sobre juegos que podrían interesarles.

In [5]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [6]:
df_games = pd.read_json('./data_limpia/output_steam_games_limpia.json', lines=True)
df_genres = pd.read_json('./data_limpia/df_dummies.json', lines=True)

In [7]:
def preparar_datos(df_games, df_genres):
    
    df_merged = df_games.merge(df_genres, on='id', how='left')
    
 
    features = ['release_date'] + list(df_genres.columns[1:])  
    
   
    scaler = StandardScaler()
    df_merged['release_date'] = scaler.fit_transform(df_merged[['release_date']])
    
    return df_merged[['id'] + features]

def muestrear_datos(df_final, frac=0.2, random_state=42):
    return df_final.sample(frac=frac, random_state=random_state)


def calcular_similitud(df_sampled, features):
    similarity_matrix = cosine_similarity(df_sampled[features].fillna(0))
    return np.nan_to_num(similarity_matrix)


def recomendacion_juego_muestreado(game_id, df_sampled, similarity_matrix, top_n=5):
  
    ids_juegos_muestreados = df_sampled['id'].unique()
    
  
    if game_id not in ids_juegos_muestreados:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    
    indice_juego = df_sampled.index[df_sampled['id'] == game_id].tolist()
    

    if not indice_juego:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    indice_juego = indice_juego[0]


    puntajes_similitud = list(enumerate(similarity_matrix[indice_juego]))
    puntajes_similitud = sorted(puntajes_similitud, key=lambda x: x[1], reverse=True)

    indices_juegos_similares = [i for i, puntaje in puntajes_similitud[1:top_n+1]]
    

    nombres_juegos_similares = df_sampled['app_name'].iloc[indices_juegos_similares].tolist()

  
    mensaje_recomendacion = f"Juegos recomendados basados en el ID del juego {game_id} - {df_sampled['app_name'].iloc[indice_juego]}:"
    
    return [mensaje_recomendacion] + nombres_juegos_similares

df_merged = preparar_datos(df_games, df_genres)
df_final = df_merged.merge(df_games[['id', 'app_name']], on='id', how='left')
df_sampled = muestrear_datos(df_final)

similarity_matrix = calcular_similitud(df_sampled, ['release_date'] + list(df_genres.columns[1:]))

#ejemplo
test_game_id = "774276"
recommendations = recomendacion_juego_muestreado(test_game_id, df_sampled, similarity_matrix, top_n=5)

print("Recomendaciones para ID de juego", test_game_id)
for rec in recommendations:
    print(rec)

ValueError: could not convert string to float: '2018-01-04'